# Using the Edge Impulse Python SDK with Weights & Biases

[![Open In Colab <](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/edgeimpulse/notebooks/blob/master/notebooks/01-python-sdk-with-wandb.ipynb)

[Weights & Biases](https://wandb.ai/) is an online framework for helping manage machine learning training, data versioning, and experiments. When running experiments for edge-focused ML projects, it can be helpful to see the required memory (RAM and ROM) along with estimated inference times of your model for your target hardware. By viewing these metrics, you can quickly gauge if your model will fit onto your target device!

Follow the code below to see how to train a simple machine learning model with different hyperparameters and log those values to the Weights & Biases dashboard.

To learn more about using the Python SDK, please see: [Edge Impulse Python SDK Overview](https://docs.edgeimpulse.com/docs/edge-impulse-python-sdk/overview.md)

In [ ]:
# If you have not done so already, install the following dependencies
!python -m pip install tensorflow wandb edgeimpulse

In [ ]:
from tensorflow import keras
import wandb
import edgeimpulse as ei

You will need to obtain an API key from an Edge Impulse project. Log into [edgeimpulse.com](https://edgeimpulse.com/) and create a new project. Open the project, navigate to **Dashboard** and click on the **Keys** tab to view your API keys. Double-click on the API key to highlight it, right-click, and select **Copy**.

![Copy API key from Edge Impulse project](../.assets/images/python-sdk-copy-ei-api-key.png)

Note that you do not actually need to use the project in the Edge Impulse Studio. We just need the API Key.

Paste that API key string in the `ei.API_KEY` value in the following cell:

In [ ]:
# Settings
ei.API_KEY = "ei_dae27..." # Change this to your Edge Impulse API key
project_name = "nodes-sweep"
labels = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
num_classes = len(labels)
savedmodel_name = "my_model"
target_device = 'cortex-m4f-80mhz'

# Define experiment hyperparameters - sweep across number of nodes
num_nodes_sweep = [8, 16, 32, 64, 128]

To use Weights and Biases, you will need to create an account on [wandb.ai](https://wandb.ai/home) and call the `wandb.login()` function. This will prompt you to log in to your account. Your credentials should be stored, which allows you to use the `wandb` package in your Python library.

In [ ]:
# Log in to Weights and Biases (will open a prompt)
wandb.login()

## Gather a dataset

We want to create a classifier that can uniquely identify handwritten digits. To start, we will use TensorFlow and Keras to train a very simple convolutional neural network (CNN) on the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, which consists of handwritten digits from 0 to 9.

In [ ]:
# Load MNIST data
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = keras.utils.normalize(x_train, axis=1)
x_test = keras.utils.normalize(x_test, axis=1)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
input_shape = x_train[0].shape

## Create an experiment

We want to vary the hyperparameters in our model and see how it affects the accuracy and predicted RAM, ROM, and inference time on our target platform. To do that, we construct a function that builds a simple model using Keras, trains the model, and computes the accuracy and loss from our holdout test set. We then use the Edge Impulse Python SDK to generate a profile of our model for our target hardware. We log the hyperparamter (number of nodes in the hidden layer), test loss, test accuracy, estimated RAM, estimated ROM, and estimated inference time (ms) to our Weights and Biases console.

In [ ]:
# Define experiment - Train and test model, log metrics
def do_experiment(num_nodes):

    # Create W&B project
    run = wandb.init(project=project_name,
                     name=f"{num_nodes}-nodes")

    # Build the model (vary number of nodes in the hidden layer)
    model = keras.Sequential([
        keras.layers.Flatten(),
        keras.layers.Dense(num_nodes, activation='relu', input_shape=input_shape),
        keras.layers.Dense(num_classes, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    model.fit(x_train, 
              y_train, 
              epochs=num_epochs)
  
    # Evaluate model
    test_loss, test_accuracy = model.evaluate(x_test, y_test)

    # Convert model to TensorFlow Lite
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()

    # Save the model in TFLite format
    with open(tflite_filename, 'wb') as f:
      f.write(tflite_model)

    # Profile model on target device
    job_resp, profile_resp = ei.model.profile(api_key=api_key, 
                                              tflite_path=tflite_filename, 
                                              device=device)

    # Log metrics
    if job_resp.success:
        print("Profiling successful. Logging.")
        wandb.log({
            'num_nodes': num_nodes,
            'test_loss': test_loss,
            'test_accuracy': test_accuracy,
            'profile_ram': profile_resp.memory.tflite.ram,
            'profile_rom': profile_resp.memory.tflite.rom,
            'inference_time_ms': profile_resp.time_per_inference_ms
        })
    else:
        print(f"Profiling unsuccessful. Error: {job_resp.error}")

    # Close run
    wandb.finish()

## Run the experiment

Now, it's time to run the experiment and log the results in Weights and Biases. Simply call our function and provide a new hyperparameter value for the number of nodes.

In [ ]:
# Perform the experiments - check your dashboard in WandB!
for num_nodes in num_nodes_sweep:
    do_experiment(num_nodes)

Head to [wandb.ai](https://wandb.ai/) and log in (if you have not already done so). Under *My projects* on the left, click on the **nodes-sweep** project. You can visualize the results of your experiments with the various charts that Weights & Biases offers!

If you would like to deploy your model to your target hardware, the Python SDK can help you with that, too. See our documentation [here](https://docs.edgeimpulse.com/docs/edge-impulse-python-sdk/overview.md).